In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras import backend as K

import os
from src import *


2023-03-18 13:01:53.919567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 13:01:54.486508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-18 13:01:54.486555: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-18 13:01:56.789090: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


CONVOLUTION_LAYERS = 5

## Load data

In [3]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
data = _df.drop(columns=["Data"]).values.transpose()
data.shape

(5, 6429)

## Process and split data

In [5]:
thr_fall, thr_rise = (-0.005270574305918364, 0.004968199728502841)

data_pipeline = DataProcess(data, test_ratio=0.2, validation_ratio=0.2, batch_size=3**CONVOLUTION_LAYERS, threshold_fall=thr_fall, threshold_rise=thr_rise, feature_to_predict_num=3)
data_pipeline.run()
_x_train, _y_train, _x_validation, _y_validation, _x_test, _y_test = data_pipeline.get_data()
_y_test_ind, _y_validation_ind = np.argmax(_y_test, axis=1), np.argmax(_y_validation, axis=1)


In [6]:
pd.DataFrame(
    {
        "x": [_x_train.shape, _x_test.shape, _x_validation.shape],
        "y": [_y_train.shape, _y_test.shape, _y_validation.shape]
    },
    index = ["train", "test", "validation"]
)


,x,y
train,"(3613, 5, 243, 2)","(3613, 3)"
test,"(1043, 5, 243, 2)","(1043, 3)"
validation,"(1043, 5, 243, 2)","(1043, 3)"


In [7]:
print("Train dataset classes value counts:")
_y_train.sum(axis=0)
# pd.DataFrame(
#     {
#         str(key): val for (key, val) in zip(*np.unique(np.argmax(_y_train, axis=1), return_counts=True))
#     },
#     index = ["count"]
# )

Train dataset classes value counts:


array([1248, 1064, 1301])

## Model

### Building

In [8]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=_x_train.shape[1:]))
for i in range(CONVOLUTION_LAYERS):
    model.add(tf.keras.layers.Conv2D(2, (1, 3), 1, padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))


2023-03-18 13:01:59.738285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-18 13:01:59.738886: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-18 13:01:59.738956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2023-03-18 13:01:59.742033: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.build(input_shape=_x_train.shape[1:])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 243, 2)         14        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 81, 2)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 81, 2)          14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 27, 2)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 27, 2)          14        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 9, 2)          0

In [11]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#               loss=f1_m,
#               metrics=f1_m
#               )
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=f1_m
              )


In [12]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      mode='min',
                                      min_delta=1e-3,
                                      patience=20,
                                      start_from_epoch=50,
                                      )
# es = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m',
#                                       mode='max',
#                                       min_delta=1e-3,
#                                       patience=20,
#                                       start_from_epoch=200,
#                                       )
mc = tf.keras.callbacks.ModelCheckpoint('Models/cdt_2/best_model.h5',
                                        monitor='val_loss',
                                        mode='min'
                                        )
# mc = tf.keras.callbacks.ModelCheckpoint('Models/cdt_2/best_model.h5',
#                                         monitor='val_f1_m',
#                                         mode='max'
#                                         )

history = model.fit(_x_train, _y_train,
                    epochs=500,
                    callbacks=[es, mc],
                    validation_data=(_x_test, _y_test)
                    )


Epoch 1/500
113/113 [==============================] - 4s 22ms/step - loss: 1.3203 - f1_m: 0.2006 - val_loss: 1.2398 - val_f1_m: 0.0326
Epoch 2/500
113/113 [==============================] - 2s 18ms/step - loss: 1.2866 - f1_m: 0.1872 - val_loss: 1.2142 - val_f1_m: 0.0214
Epoch 3/500
113/113 [==============================] - 2s 18ms/step - loss: 1.2569 - f1_m: 0.1677 - val_loss: 1.1914 - val_f1_m: 0.0109
Epoch 4/500
113/113 [==============================] - 2s 20ms/step - loss: 1.2310 - f1_m: 0.1582 - val_loss: 1.1729 - val_f1_m: 0.0113
Epoch 5/500
113/113 [==============================] - 2s 17ms/step - loss: 1.2085 - f1_m: 0.1448 - val_loss: 1.1574 - val_f1_m: 0.0070
Epoch 6/500
113/113 [==============================] - 2s 17ms/step - loss: 1.1889 - f1_m: 0.1343 - val_loss: 1.1444 - val_f1_m: 0.0053
Epoch 7/500
113/113 [==============================] - 2s 17ms/step - loss: 1.1722 - f1_m: 0.1238 - val_loss: 1.1340 - val_f1_m: 0.0036
Epoch 8/500
113/113 [===========================

In [ ]:
# model.save("Models/cdt_2")

### Evaluating

### Valitation

In [13]:
model.evaluate(_x_validation, _y_validation)

33/33 [==============================] - 0s 9ms/step - loss: 1.1096 - f1_m: 0.0000e+00


[1.109598994255066, 0.0]

In [14]:
pred_validation = model.predict(_x_validation)
pred_validation_ind = np.argmax(pred_validation, axis=1)

33/33 [==============================] - 1s 9ms/step


In [15]:
print("Validation correct:", np.sum(pred_validation_ind == _y_validation_ind))
print("Validation accuracy:", np.sum(pred_validation_ind == _y_validation_ind)/pred_validation_ind.shape[0])

Validation correct: 311
Validation accuracy: 0.2981783317353787


In [16]:
confusion_matrix(_y_validation_ind, pred_validation_ind)

array([[ 93,   1, 189],
       [143,   1, 309],
       [ 90,   0, 217]])

### Test

In [17]:
model.evaluate(_x_test, _y_test)

33/33 [==============================] - 0s 10ms/step - loss: 1.1018 - f1_m: 0.0000e+00


[1.1018249988555908, 0.0]

In [18]:
pred_test = model.predict(_x_test)
pred_test_ind = np.argmax(pred_test, axis=1)

33/33 [==============================] - 0s 9ms/step


In [19]:
print("Test correct:", np.sum(pred_test_ind == _y_test_ind))
print("Test accuracy:", np.sum(pred_test_ind == _y_test_ind)/pred_test_ind.shape[0])

Test correct: 336
Test accuracy: 0.3221476510067114


In [20]:
confusion_matrix(_y_test_ind, pred_test_ind)

array([[ 63,   5, 286],
       [ 62,   4, 283],
       [ 68,   3, 269]])

# Draft

In [ ]:
a = np.broadcast_to(np.arange(10).reshape((1, 1, -1, 1)), (50, 1, 10, 2))
